<a href="https://colab.research.google.com/github/felanbi/Projet-OC-Categoriser-automatiquement-des-questions/blob/main/P5_01_notebookexploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
path_queries = 'drive/MyDrive/data/'

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import IPython.display
import os
import nltk
import re
import string
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import FreqDist

In [3]:
%load_ext autoreload
%autoreload 2